In [ ]:
pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
pip install annoy

SyntaxError: invalid syntax (<ipython-input-9-a6059851aee1>, line 1)

In [ ]:
pip install opendatasets

In [ ]:
import opendatasets as od
dataset = 'https://www.kaggle.com/datasets/shaunthesheep/microsoft-catsvsdogs-dataset/data'

In [ ]:
od.download(dataset)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: harshitsandhu
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/shaunthesheep/microsoft-catsvsdogs-dataset


100%|██████████| 788M/788M [00:52<00:00, 15.8MB/s]


In [ ]:
import os
from torchvision import models, transforms
from annoy import AnnoyIndex
import torch.nn as nn
from PIL import Image
image_folder = '/content/microsoft-catsvsdogs-dataset/PetImages/Dog'
images = os.listdir(image_folder)

weigths = models.ResNet18_Weights.IMAGENET1K_V1
model = models.resnet18(weights=weigths)
model.fc = nn.Identity()
model.eval()

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])



In [ ]:
from annoy import AnnoyIndex

annoy_index = AnnoyIndex(512, 'angular')

for i in range(len(images)):
  image_path = os.path.join(image_folder, images[i])
  try:
    image = Image.open(image_path)
    input_tensor = transform(image).unsqueeze(0)

    if input_tensor.size()[1] == 3:
      output_tensor = model(input_tensor)
      annoy_index.add_item(i, output_tensor[0])

    if i%100==0:
      print(f'Processed {i} Images.')
  except IOError as e:
    print(f"Error processing image {image_path}: {e}")

annoy_index.build(10)
annoy_index.save('dog_index.ann')

Processed 0 Images.
Processed 100 Images.
Processed 200 Images.
Processed 300 Images.
Processed 400 Images.
Processed 500 Images.


/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Processed 600 Images.
Processed 700 Images.
Processed 800 Images.
Processed 900 Images.
Processed 1000 Images.
Processed 1100 Images.
Processed 1200 Images.
Processed 1300 Images.
Processed 1400 Images.
Processed 1500 Images.
Processed 1600 Images.
Processed 1700 Images.
Processed 1800 Images.
Processed 1900 Images.
Processed 2000 Images.
Processed 2100 Images.
Error processing image /content/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg: cannot identify image file '/content/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg'
Processed 2200 Images.
Processed 2300 Images.
Processed 2400 Images.
Processed 2500 Images.
Processed 2600 Images.
Processed 2700 Images.
Processed 2800 Images.
Processed 2900 Images.
Processed 3000 Images.
Processed 3100 Images.
Processed 3200 Images.
Processed 3300 Images.
Processed 3400 Images.
Processed 3500 Images.
Processed 3600 Images.
Processed 3700 Images.
Processed 3800 Images.
Processed 3900 Images.
Processed 4000 Images.
Processed 4100 Images.

True

In [ ]:
from pathlib import WindowsPath
from PIL import Image, ImageDraw
annoy_index.load('dog_index.ann')
image_grid = Image.new('RGB', size=(1000, 1000))

for i in range(len(images)):
    image_path = os.path.join(image_folder, images[i])
    image = Image.open(image_path)
    input_tensor = transform(image).unsqueeze(0)

    if input_tensor.size()[1] == 3:
      output_tensor = model(input_tensor)
      nns = annoy_index.get_nns_by_vector(output_tensor[0], 24)
      image = image.resize((200, 200))
      image_draw = ImageDraw.Draw(image)
      image_draw.rectangle((0, 0, 199, 199), outline='red',width = 8)
      image_grid.paste(image,((0,0)))

      for j in range(24):
        search_image = Image.open(os.path.join(image_folder, images[nns[j]]))
        search_image = search_image.resize((200, 200))
        image_grid.paste(search_image, (((j+1) % 5) * 200, ((j+1) // 5) * 200))
      image_grid.save(f'ImageDump/image_{i}.png')

/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


KeyboardInterrupt: 